# Leitura de .hdf5 e criação de mapa

## Bibliotecas necessárias

Além das bibliotecas utilizadas para leitura, manipulação e visualização de dados, foi preciso importar também a biblioteca `warnings` e utilizar seu módulo `resetwarnings()` para ignorar um problema da versão 2.7.1 da `h5py` que ocorre nas versões do Python3.

Utilizações das bibliotecas importadas:
* `h5py`: leitura de arquivos no formato hdf5
* `numpy`: manipulação de dados
* `plotly`: criação de mapa

In [2]:
#Para ignorar warning da biblioteca h5py versão 2.7.1 (https://github.com/quantumlib/OpenFermion/issues/175):
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
import h5py
warnings.resetwarnings()

import numpy as np

from plotly.offline import init_notebook_mode, iplot
init_notebook_mode(connected=True) # initiate notebook for offline plot
import plotly.graph_objs as go
import plotly.tools as tls

## Leitura do arquivo .hdf5
Primeiro foram verificados os conjuntos de dados existentes no arquivo, com a utilização do comando `keys()`, e escolhidos três para o desenvolvimento do programa: *aerosol_optical_thickness_550_ocean*, que contém dados da espessura óptica do aerossol estimada em 550 nm, acima do oceano; e *latitude* e *longitude*, que correspondem às coordenadas geográficas referênctes a essas estimativas.

Depois, com o comando `attrs.keys()` foram verificados os atributos disponíveis para o conjunto de dados, para posterior utilização de seu nome, identificado em `long_name`.

Também foram necessárias algumas manipulações. Por exemplo, atributos do tipo string precisam ser decodificadas para UTF-8 (Python3), e os valores indefinidos no conjunto de dados foram preenchidos com valor do tipo `NaN`. Por último, fez-se uma média dos valores disponíveis para cada ponto, que foram utilizados na elaboaração do mapa.

In [3]:
#Nome do arquivo .hdf5
file_name = 'DeepBlue-SeaWiFS-1.0_L3_20100101_v004-20130604T131317Z.h5'

#Fazer leitura do arquivo com a biblioteca h5py:
with h5py.File(file_name, mode='r') as f:
    #Para saber os conjuntos de dados disponíveis no arquivo:
    #print(list(f.keys()))

    #Arrays com dados da espessura óptica do aerossol estimada em 550 nm, acima do oceano, e longitude e latitude dos dados:
    data = f['aerosol_optical_thickness_550_ocean'][:]
    latitude = f['latitude'][:]
    longitude = f['longitude'][:]
    
    #Para saber os atributos do conjunto de dados:
    #print(list(f['aerosol_optical_thickness_550_ocean'].attrs.keys()))
    
    #Nome do conjunto de dados - Atributos do tipo string precisam ser decodificadas para UTF-8 (python3):
    long_name = f['aerosol_optical_thickness_550_ocean'].attrs['long_name'].decode()
    
    #Alterar valores indefinidos no conjunto de dados para serem do tipo NaN
    _FillValue = f['aerosol_optical_thickness_550_ocean'].attrs['_FillValue']
    data[data == _FillValue] = np.nan
    data = np.ma.masked_array(data, np.isnan(data))
    data_ = []
    for i in range(len(data)):
        data_.append(np.mean(data[i]))

## Criação do mapa
Para melhor apresentar as informações contidas nos dados, foi criado um mapa utilizando a bibliotexa `plotly`.

In [4]:
    scl = [[0,"rgb(150,0,90)"],[0.125,"rgb(0, 0, 200)"],[0.25,"rgb(0, 25, 255)"],[0.375,"rgb(0, 152, 255)"],[0.5,"rgb(44, 255, 150)"],[0.625,"rgb(151, 255, 0)"],[0.75,"rgb(255, 234, 0)"],[0.875,"rgb(255, 111, 0)"],[1,"rgb(255, 0, 0)"]]
    
    trace = [ dict(
        lat = latitude,
        lon = longitude,
        text = data_,
        marker = dict(
            color = data_,
            colorscale = scl,
            reversescale = True,
            opacity = 0.7,
            size = 3,
            colorbar = dict(
                thickness = 10,
                titleside = "right",
                outlinecolor = "rgba(68, 68, 68, 0)",
                ticks = "outside",
                ticklen = 3,
                showticksuffix = "last",
                dtick = 0.1
            ),
        ),
        type = 'scattergeo'
    ) ]

    layout = dict(
        geo = dict(
            showland = True,
            landcolor = "rgb(212, 212, 212)",
            subunitcolor = "rgb(255, 255, 255)",
            countrycolor = "rgb(255, 255, 255)",
            showlakes = True,
            lakecolor = "rgb(255, 255, 255)",
            showsubunits = True,
            showcountries = True,
            resolution = 50,
        ),
        title = long_name,
    )
  
    fig = {'data':trace, 'layout':layout }
    iplot(fig, filename="aerosol_map") # use plotly.offline.iplot for offline plot

Como podemos perceber do mapa gerado, os dados existentes nesse conjunto de dados cobre apenas uma pequena parte do planeta. Pode ser que a inserção de outros conjuntos do mesmo arquivo ajudem a complementá-lo, ficando como sugestão para próximos testes.